In [3]:
import pandas as pd
from IPython.display import Markdown, display
from openbb import obb
obb.user.preferences.output_type = "dataframe"
STOCK_DATA_STORE = "stocks.h5"
FUTURES_DATA_STORE = "futures.h5"
ticker = "SPY"
root = "CL"

In [5]:
STOCKS_DATA_STORE = "stocks.h5"
FUTURES_DATA_STORE = "futures.h5"

In [7]:
# Loading the data for SPY
spy_equity = obb.equity.price.historical(
    ticker,
    start_date="2021-01-01",
    provider="yfinance"
)
spy_chains = obb.derivatives.options.chains(
    ticker,
    provider="cboe"
)

spy_expirations = ( spy_chains.expiration.astype(str).unique().tolist() )

spy_historic = (
    obb
    .equity
    .price
    .historical(
        ticker + spy_expirations[-10].replace("-", "")[2:] + "c" + "00400000",
        start_date = "2021-01-01",
        provider="yfinance"
    )
)

In [8]:
# Store the data in the HDF5 file
with pd.HDFStore(STOCKS_DATA_STORE) as store:
    store.put("equities/spy/stock_prices", spy_equity)
    store.put("equities/spy/options_prices", spy_historic)
    store.put("equities/spy/chains", spy_chains)

C:\Users\tsebo\AppData\Local\Temp\ipykernel_12264\2959147672.py:5: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->Index(['underlying_symbol', 'contract_symbol', 'expiration', 'option_type',
       'last_trade_time', 'tick'],
      dtype='object')]

  store.put("equities/spy/chains", spy_chains)


In [11]:
# Read the data from the HDF5 file into DataFrames
with pd.HDFStore(STOCKS_DATA_STORE) as store:
    spy_prices = store["equities/spy/stock_prices"]
    spy_options = store["equities/spy/options_prices"]
    spy_chains = store["equities/spy/chains"]

In [ ]:
# Iterate through e-mini futures expirations, storing the historical data for each one in the 
# different file path
with pd.HDFStore(FUTURES_DATA_STORE) as store:
    for i in range(24, 31):
        expiry = f"20{i}-12"
        df = obb.derivatives.futures.historical(
            symbol=[root],
            expiry=expiry,
            start_date="2021-01-01",
            provider="yfinance"
        )
        df.rename(
            columns={
                "close": expiry
            },
            inplace=True
        )
        prices = df[expiry]
        stores.put(f'futures/{root}/{expiry}', prices)

In [ ]:
with pd.HDFStore(FUTURES_DATA_STORE) as store:
    es_prices = store[f"futures/{root}/2025-12"]